In [6]:
import numpy as np

softmax_outputs = np.array([[0.7, 0.1, 0.2],
                           [0.1, 0.5, 0.4],
                           [0.02, 0.9, 0.08]])

class_targets = np.array([[1, 0, 0],
                         [0, 1, 0],
                         [0, 1, 0]])

In [1]:
# CATEGORICAL CROSS-ENTROPY LOSS

class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

In [3]:
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        #clip data to prevent division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        #probability for target values
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
            range(samples),
            y_true
            ]
        # mask values for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum( 
            y_pred_clipped * y_true,
            axis=1
            )
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
            

In [7]:
loss_function = Loss_CategoricalCrossEntropy()
loss = loss_function.calculate(softmax_outputs, class_targets)
print(loss)

0.38506088005216804


In [8]:
# COMBINING THE ENTIRE CODE 

In [9]:
import nnfs
from nnfs.datasets import spiral_data

In [10]:
nnfs.init()

In [12]:
# DENSE LAYER
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        #initalise weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

#RELU ACTIVATION
class Activation_RELU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

# SOFTMAX ACTIVATION
class Activation_Softmax:
    def forward(self, inputs):
        # unnormalized Probabilites
        exp_values = np.exp(inputs - np.max(inputs, axis=1, 
                                            keepdims=True))
        
        # Normalize the values
        probabilities = exp_values / np.sum(exp_values, axis=1,
                                    keepdims=True)
        
        self.output = probabilities

# COMMON LOSS CLASS
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

# CROSS ENTROPY LOSS
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        #clip data to prevent division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        #probability for target values
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
            range(samples),
            y_true
            ]
        # mask values for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum( 
            y_pred_clipped * y_true,
            axis=1
            )
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

# CREATE DATASET 
X, y = spiral_data(samples=100, classes=3)

#create dense layer (2 input features, 3 output values)
dense1 = Layer_Dense(2,3)

#using RELU activation
activation1 = Activation_RELU()

#second dense layer with 3 input and features(as this the output of previous layer) and 3 output values
dense2 = Layer_Dense(3,3)

#using softmax acttvation
activation2 = Activation_Softmax()

#create loss function
loss_function = Loss_CategoricalCrossEntropy()

#perform forward pass
dense1.forward(X)

activation1.forward(dense1.output)

dense2.forward(activation1.output)

activation2.forward(dense2.output)

print(activation2.output[:5])

loss = loss_function.calculate(activation2.output, y)

print(f'Loss :{loss}')

[[0.33333334 0.33333334 0.33333334]
 [0.3333332  0.3333332  0.33333364]
 [0.3333329  0.33333293 0.3333342 ]
 [0.3333326  0.33333263 0.33333477]
 [0.33333233 0.3333324  0.33333528]]
Loss :1.0986104011535645


In [14]:
# ACCURACY CALCULATION

# probabilties of 3 samples
softmax_outputs = np.array([[0.7, 0.2, 0.1],
                            [0.5, 0.1, 0.4],
                            [0.02, 0.9, 0.08]])
#targets 
class_targets = np.array([0,1,1])

#Calculate values along second axis (axis of index 1)
predictions = np.argmax(softmax_outputs, axis = 1 )
# If targets are one-hot encoded - convert them
if len (class_targets.shape) == 2:
    class_targets = np.argmax(class_targets, axis = 1 )
# True evaluates to 1; False to 0
accuracy = np.mean(predictions == class_targets)
print ( 'acc:' , accuracy)

acc: 0.6666666666666666


In [15]:
# Calculate accuracy from output of activation2 and targets
# calculate values along first axis
predictions = np.argmax(activation2.output, axis = 1 )
if len (y.shape) == 2 :
    y = np.argmax(y, axis = 1 )
accuracy = np.mean(predictions == y)
# Print accuracy
print ( 'acc:' , accuracy)

acc: 0.34
